In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#!pip install --upgrade pip
!pip install datasets torch torchvision matplotlib datasets
!pip install -U transformers==4.40.2 accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 36.8 MB/s eta 0:00:00
 

In [3]:
!pip install huggingface_hub
#!huggingface-cli login

In [4]:
import os
#os.chdir("content")
!ls
contents = os.listdir('drive/MyDrive/Colab Notebooks')
print(contents)
!pwd

drive  sample_data
['997128_dataset 2024-11-11 18-10-43', 'Copy of Welcome To Colab', 'DataEngineeringProject', 'Masinõppe_projekti_testimine.ipynb']
/content


In [5]:
import os
import torch
from pycocotools.coco import COCO
from datasets import load_dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from PIL import Image
import matplotlib.pyplot as plt
from datasets import load_dataset
from sklearn.model_selection import train_test_split

annotations_path = "drive/MyDrive/Colab Notebooks/997128_dataset 2024-11-11 18-10-43/annotations/instances.json"
images_path = "drive/MyDrive/Colab Notebooks/997128_dataset 2024-11-11 18-10-43/images/"


coco = COCO(annotations_path)

image_ids = list(coco.imgs.keys())
train_ids, test_ids = train_test_split(image_ids, test_size=0.2, random_state=42)

def get_coco_data(coco, image_ids):
    data = []
    for img_id in image_ids:
        image_info = coco.imgs[img_id]
        annotations = coco.loadAnns(coco.getAnnIds(imgIds=[img_id]))
        data.append({"image_id": img_id, "image_info": image_info, "annotations": annotations})
    return data

train_dataset = get_coco_data(coco, train_ids)
test_dataset = get_coco_data(coco, test_ids)

loading annotations into memory...
Done (t=1.06s)
creating index...
index created!


In [6]:
from transformers import Owlv2Processor, Owlv2ForObjectDetection

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

processor = Owlv2Processor.from_pretrained("google/owlv2-base-patch16-ensemble", force_download=True, cache_dir='./cache')
model = Owlv2ForObjectDetection.from_pretrained("google/owlv2-base-patch16-ensemble", force_download=True, cache_dir='./cache').to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/620M [00:00<?, ?B/s]

In [7]:
help(model.forward)

Help on method forward in module transformers.models.owlv2.modeling_owlv2:

forward(input_ids: torch.Tensor, pixel_values: torch.FloatTensor, attention_mask: Optional[torch.Tensor] = None, output_attentions: Optional[bool] = None, output_hidden_states: Optional[bool] = None, return_dict: Optional[bool] = None) -> transformers.models.owlv2.modeling_owlv2.Owlv2ObjectDetectionOutput method of transformers.models.owlv2.modeling_owlv2.Owlv2ForObjectDetection instance
    The [`Owlv2ForObjectDetection`] forward method, overrides the `__call__` special method.
    
    <Tip>
    
    Although the recipe for forward pass needs to be defined within this function, one should call the [`Module`]
    instance afterwards instead of this since the former takes care of running the pre and post processing steps while
    the latter silently ignores them.
    
    </Tip>
    
    Args:
        pixel_values (`torch.FloatTensor` of shape `(batch_size, num_channels, height, width)`):
            Pixel val

In [14]:
def train_model(model, processor, train_data, images_path, epochs=3, lr=1e-4, batch_size=3):
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    model.train()

    for epoch in range(epochs):
        print(f"\nStarting Epoch {epoch + 1}/{epochs}")

        for idx in range(0, len(train_data), batch_size):
            batch_data = train_data[idx:idx + batch_size]
            print(f"Processing batch {idx // batch_size + 1}/{len(train_data) // batch_size + 1}")

            images = []
            image_ids= []
            for item in batch_data:
                image_id = item['image_id']
                image_ids.append(image_id)
                image_file = os.path.join(images_path, item['image_info']['file_name'])
                print(f"Loading image from: {image_file}")
                image = Image.open(image_file).convert("RGB")
                images.append(image)

            inputs = processor(images=images, return_tensors="pt").to(device)
            print("Processor output keys:", inputs.keys())

            outputs = model(pixel_values=inputs['pixel_values'], input_ids=image_ids)
            print("Model outputs:", outputs)

            loss = outputs.loss
            print(f"Loss computed: {loss.item()}")

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

    return model

def test_model(model, processor, test_data, images_path):
    model.eval()
    results = []

    images = []
    image_ids = []

    for item in test_data:
        image_id = item['image_id']
        image_ids.append(image_id)
        image_file = os.path.join(images_path, item['image_info']['file_name'])
        print(f"Loading test image from: {image_file}")
        image = Image.open(image_file).convert("RGB")
        images.append(image)

    inputs = processor(images=images, return_tensors="pt").to(device)
    print("Inputs processed successfully:", inputs)

    outputs = model(pixel_values=inputs['pixel_values'], input_ids=image_ids)
    print("Model outputs received:", outputs)

    boxes, scores = outputs["boxes"], outputs["scores"]

    for i, (image, box, score) in enumerate(zip(images, boxes, scores)):
        plt.imshow(image)
        for b, s in zip(box, score):
            x1, y1, x2, y2 = b
            plt.gca().add_patch(plt.Rectangle((x1, y1), x2 - x1, y2 - y1, fill=False, color="red", linewidth=2))
            plt.text(x1, y1, f"{s:.2f}", color="red")
        plt.show()

        results.append((image_file, box, score))

    return results

trained_model = train_model(model, processor, train_dataset, images_path)

test_results = test_model(trained_model, processor, test_dataset, images_path)



Starting Epoch 1/3
Processing batch 1/101
Loading image from: drive/MyDrive/Colab Notebooks/997128_dataset 2024-11-11 18-10-43/images/041653.jpg
Loading image from: drive/MyDrive/Colab Notebooks/997128_dataset 2024-11-11 18-10-43/images/041728.jpg
Loading image from: drive/MyDrive/Colab Notebooks/997128_dataset 2024-11-11 18-10-43/images/041623.jpg
Processor output keys: dict_keys(['pixel_values'])


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.74 GiB. GPU 0 has a total capacity of 14.75 GiB of which 1.38 GiB is free. Process 3878 has 13.37 GiB memory in use. Of the allocated memory 12.80 GiB is allocated by PyTorch, and 452.29 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import matplotlib.pyplot as plt

# Kuvame tulemuse
plt.figure(figsize=(10, 10))  # Suurendame pildi kuvamise akent
plt.imshow(image_with_boxes)
plt.axis('off')  # Ei kuvata telgi koordinaate
plt.show()